In [45]:
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials
import gspread
import pandas as pd
import numpy as np
import json
import sqlite3
import datetime
from dateutil.relativedelta import relativedelta


HALL_NAME = "第一プラザ狭山店"
HALL_NAME = "パラッツォ川越店"
HALL_NAME = "パールショップともえ川越店"
HALL_NAME = "EXA FIRST"

DB_PATH = r"C:\python\dataOnline\anaslo_02\db\anaslo_02.db"

# spreadSheet_ids = {
#     "EXA FIRST": "10-B_vV1pvUzXmvGAiHhODGJgCloOsAmqSO9HvXpk_T8",
    # "パラッツォ川越店": "179nJF0NvLng7xPKsd_NX2pJBXsDNsO8SJhOvUAvFk2I",
    # "第一プラザ狭山店": "1IVb2Woq3n_PDZP87LdW9NpFP3Z6LeyQtONCkx_fWIq4",
    # "パールショップともえ川越店": "1i70joJ27Hs7inS-D89z9YMSJO1aRvaBeeWn0n9xpktY",
    # }

# 検索キーワードよりホール名取得
# SPREADSHEET_ID = spreadSheet_ids[HALL_NAME]

# スプレッドシート認証設定
# scope = [
#     "https://spreadsheets.google.com/feeds",
#     "https://www.googleapis.com/auth/drive",
# ]
# jsonf = r"C:\python\dataOnline\anaslo_02\json\spreeadsheet-347321-ff675ab5ccbd.json"
# creds = ServiceAccountCredentials.from_json_keyfile_name(jsonf, scope)
# client = gspread.authorize(creds)
# spreadsheet = client.open_by_key(SPREADSHEET_ID)

# # Table name 取得
conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;")
tables = cursor.fetchall()
# print(tables)

cursor.execute(
    "SELECT hall_id, name FROM halls WHERE name LIKE ?", ("%" + HALL_NAME + "%",)
)
results = cursor.fetchall()

# 結果表示
if results:
    for hall_id, hall_name in results:
        # print(f" - hall_name: {hall_name}, hall_id: {hall_id}")
        print(f"🔍 '{HALL_NAME}' を含むホール名が見つかりました。")
else:
    print(f"❌ '{HALL_NAME}' を含むホール名は見つかりませんでした。")

query = """
-- 出玉データにホール名と機種名を結合して取得
SELECT
    r.*, 
    h.name AS hall_name,     -- ホール名を追加
    m.name AS model_name     -- 機種名を追加
FROM results r
JOIN halls h ON r.hall_id = h.hall_id  -- ホールと結合
JOIN models m ON r.model_id = m.model_id  -- 機種と結合
WHERE h.name = ?  -- 指定ホールのみ
AND m.name LIKE '%ジャグラー%'  -- ジャグラー系機種に限定
ORDER BY r.date DESC, r.unit_no ASC;
"""

df = pd.read_sql_query(query, conn, params=(hall_name,))
conn.close()

# ブドウシミュレーター
def grape_calculator_myfive(game, bb, rb, medals, cherry=True):
    bb_medals = 239.25
    rb_medals = 95.25
    replay_rate = 0.411
    if cherry:
        cherry_rate_high = 0.04228
    else:
        cherry_rate_high = 0.05847
    denominator_inner = (-medals - (game*3 - (bb*bb_medals + rb*rb_medals + game*replay_rate + game*cherry_rate_high))) / 8
    grape_rate = (game / denominator_inner) - ((game / denominator_inner) * 2)
    
    return grape_rate

def assign_area(unit_no, json_file_path):
    with open(json_file_path, "r", encoding="utf-8") as f:
        area_map = json.load(f)
    for rule in area_map:
        if rule["start"] <= unit_no <= rule["end"]:
            return rule["area"]
    return "その他"

df["date"] = pd.to_datetime(df["date"])
df.drop(columns=["result_id", "hall_id", "model_id"], inplace=True)
df = df[["hall_name", "date", "model_name", "unit_no", "game", "BB", "RB", "medals"]]
df["BB_rate"] = (df["game"] / df["BB"]).round(1)
df["RB_rate"] = (df["game"] / df["RB"]).round(1)
df["Grape_rate"] = grape_calculator_myfive(df["game"], df["BB"], df["RB"], df["medals"], cherry=True).round(2)
df["Total_rate"] = (df["game"] / (df["BB"] + df["RB"])).round(1)
df["month"] = df["date"].dt.strftime("%Y-%m")
df["day"] = df["date"].dt.day
df["weekday"] = df["date"].dt.weekday
df["unit_last"] = df["unit_no"].astype(str).str[-1]

JSON_FILE_PATH = r"C:\python\dataOnline\anaslo_02\json\exa_area_map.json"
df["area"] = df["unit_no"].apply(lambda x: assign_area(x, JSON_FILE_PATH))

df = df.replace([np.inf, -np.inf], np.nan)
df = df.fillna(0)

print(f'データサイズ: {df.shape[0]} x {df.shape[1]}')
print(f'以下の日付のデータが含まれています')
print(df.date.unique()[0])
model_list = list(df["model_name"].unique())
print(f'以下のモデルが含まれています')
for i, model in enumerate(model_list):
    print(f'{i}: {model}', end=", ")
    
df.head()

🔍 'EXA FIRST' を含むホール名が見つかりました。
データサイズ: 58672 x 17
以下の日付のデータが含まれています
2025-04-24 00:00:00
以下のモデルが含まれています
0: ゴーゴージャグラー3, 1: マイジャグラーV, 2: ファンキージャグラー2, 3: アイムジャグラーEX-TP, 4: ミスタージャグラー, 5: ウルトラミラクルジャグラー, 6: ハッピージャグラーVIII, 7: ジャグラーガールズ, 

,hall_name,date,model_name,unit_no,game,BB,RB,medals,BB_rate,RB_rate,Grape_rate,Total_rate,month,day,weekday,unit_last,area
0,EXA FIRST,2025-04-24,ゴーゴージャグラー3,1001,3613,11,8,-1088,328.5,451.6,6.12,190.2,2025-04,24,3,1,a
1,EXA FIRST,2025-04-24,ゴーゴージャグラー3,1002,5375,28,18,1418,192.0,298.6,6.42,116.8,2025-04,24,3,2,a
2,EXA FIRST,2025-04-24,ゴーゴージャグラー3,1003,957,5,2,174,191.4,478.5,6.25,136.7,2025-04,24,3,3,a
3,EXA FIRST,2025-04-24,ゴーゴージャグラー3,1004,3202,17,6,679,188.4,533.7,6.11,139.2,2025-04,24,3,4,a
4,EXA FIRST,2025-04-24,ゴーゴージャグラー3,1005,5347,20,14,-253,267.4,381.9,5.90,157.3,2025-04,24,3,5,a


In [73]:
df[df["model_name"] == "ハッピージャグラーVIII"]

,hall_name,date,model_name,unit_no,game,BB,RB,medals,BB_rate,RB_rate,Grape_rate,Total_rate,month,day,weekday,unit_last,area
134,EXA FIRST,2025-04-24,ハッピージャグラーVIII,1137,1612,5,2,-538,322.4,806.0,5.91,230.3,2025-04,24,3,7,r
135,EXA FIRST,2025-04-24,ハッピージャグラーVIII,1138,2767,8,7,-715,345.9,395.3,5.90,184.5,2025-04,24,3,8,r
136,EXA FIRST,2025-04-24,ハッピージャグラーVIII,1139,1109,3,2,-479,369.7,554.5,6.17,221.8,2025-04,24,3,9,r
137,EXA FIRST,2025-04-24,ハッピージャグラーVIII,1140,889,2,1,-426,444.5,889.0,5.63,296.3,2025-04,24,3,0,r
138,EXA FIRST,2025-04-24,ハッピージャグラーVIII,1141,959,1,2,-897,959.0,479.5,6.88,319.7,2025-04,24,3,1,r
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29934,EXA FIRST,2024-10-07,ハッピージャグラーVIII,1146,8211,36,34,2921,228.1,241.5,5.48,117.3,2024-10,7,0,6,s
29935,EXA FIRST,2024-10-07,ハッピージャグラーVIII,1147,8162,33,35,2091,247.3,233.2,5.61,120.0,2024-10,7,0,7,s
29936,EXA FIRST,2024-10-07,ハッピージャグラーVIII,1148,8266,38,14,912,217.5,590.4,5.73,159.0,2024-10,7,0,8,s
29937,EXA FIRST,2024-10-07,ハッピージャグラーVIII,1149,7591,37,27,2718,205.2,281.1,5.71,118.6,2024-10,7,0,9,s


In [ ]:
# 各種ピボットテーブル
def new_func(df, model_name, start_date, end_date):
    df_filtered = df.copy()
    df_filtered = df_filtered[df_filtered["model_name"] == model_name]
    df_filtered = df_filtered[(df_filtered["date"].dt.date >= start_date) & (df_filtered["date"].dt.date <= end_date)]
    pivot_targets = ["game", "RB_rate", "Total_rate"]
    pivot_results = {}
    for col in pivot_targets:
        table = df_filtered.pivot_table(
        index=["model_name", "unit_no"],
        columns="date",
        values=col,
        aggfunc="sum",
    )
        # 日付列を反転・スライス
        pivot_results[col] = table.iloc[:, ::-1]

    game = pivot_results["game"]
    rb_rate = pivot_results["RB_rate"]
    total_rate = pivot_results["Total_rate"]

    # labeled_tables = [("GAME", game), ("RB_RATE", rb_rate), ("TOTAL_RATE", total_rate)]

    # ラベルを MultiIndex に付ける
    # for label, df_table in labeled_tables:
    #     df_table.columns = pd.MultiIndex.from_product([[label], df_table.columns])

    # display(game.head(2))
    # display(rb_rate.head(2))
    # display(total_rate.head(2))

    # 列を交互に整列して統合・NaN除去
    # interleaved_cols = [col for pair in zip(
    #     game.columns,
    #     rb_rate.columns,
    #     total_rate.columns,
    # ) for col in pair]
    # merged = pd.concat([game, rb_rate, total_rate], axis=1)[interleaved_cols]
    
    return game, rb_rate, total_rate

start_date = datetime.date(2024, 4, 25)
end_date = datetime.date(2024, 5, 5)
game, rb_rate, total_rate = new_func(df, model, start_date, end_date)

In [92]:
model_list = [
    "マイジャグラーV",
    "ゴーゴージャグラー3",
    "アイムジャグラーEX-TP",
    "ファンキージャグラー2",
    # "ミスタージャグラー",
    # "ウルトラミラクルジャグラー",
    # "ハッピージャグラーVIII",
    # "ジャグラーガールズ",
]


start_date = datetime.date(2024, 4, 25)
end_date = datetime.date(2024, 5, 5)

df_filtered = df.copy()
merged_models = pd.DataFrame()
for model in list(model_list):
    print(model)
    empty_index = pd.MultiIndex.from_tuples([("", "")], names=game.index.names)
    empty_row = pd.DataFrame([[""] * game.shape[1]], index=empty_index, columns=game.columns)
    game, rb_rate, total_rate = new_func(df_filtered, model, start_date, end_date)
    merged_model = pd.concat([game, empty_row, rb_rate, empty_row, total_rate, empty_row], axis=0)
    # display(merged_model)
    merged_models = pd.concat([merged_models, merged_model, empty_row])

merged_models = merged_models[~merged_models.iloc[:, 0].isna()]
merged_models.to_csv(f"{HALL_NAME}_GW_data.csv")
    
merged_models

マイジャグラーV
ゴーゴージャグラー3
アイムジャグラーEX-TP
ファンキージャグラー2


date                2024-05-05 2024-05-04 2024-05-03 2024-05-02 2024-05-01  \
model_name  unit_no                                                          
マイジャグラーV    1001        7897.0     8802.0     5811.0     8280.0     8472.0   
            1002        5562.0     7093.0     9127.0     6882.0     5860.0   
            1003        6539.0     7080.0     5133.0     8160.0     8275.0   
            1004        6860.0     6365.0     7624.0     2007.0     6090.0   
            1005        5044.0     6112.0     7463.0     4374.0     6024.0   
...                        ...        ...        ...        ...        ...   
ファンキージャグラー2 1062         155.7      173.6      157.4      146.1      223.4   
            1063         153.4      126.6      182.6      137.3      125.2   
            1064         130.2      207.0      164.5      199.8      184.8   
                                                                             
                                                                             

date                2024-04-30 2024-04-29 2024-04-28 2024-04-27 2024-04-26  \
model_name  unit_no                                                          
マイジャグラーV    1001        4683.0     6609.0     5746.0     7247.0     7751.0   
            1002        5089.0     8288.0     7922.0     6399.0     8477.0   
            1003        3776.0     5138.0     5375.0     6289.0     6457.0   
            1004        8142.0     6875.0     3404.0     9876.0     7998.0   
            1005        4123.0     8952.0     8924.0     4997.0     6588.0   
...                        ...        ...        ...        ...        ...   
ファンキージャグラー2 1062         129.7      222.3      198.7      186.0      158.7   
            1063         160.0      140.4      165.9      181.5      125.9   
            1064         149.8      152.4      198.5      133.9      154.0   
                                                                             
                                                                             

date                2024-04-25  
model_name  unit_no             
マイジャグラーV    1001        8488.0  
            1002        9187.0  
            1003        6568.0  
            1004        5140.0  
            1005        9121.0  
...                        ...  
ファンキージャグラー2 1062         120.6  
            1063         166.1  
            1064         171.2  
                                
                                

[393 rows x 11 columns]